# 02 Initial steps

## Six-pairs

Stedman's pattern of alternating quick and slow sixes makes it awkward to work with. Let's instead combine pairs of sixes together. Each six-pair can have the following calling structure:

|   | Call 1 | Call 2 | Row (from rounds) | Permutation             |
| -:|:------:|:------:|:-----------------:|:----------------------- |
| 1 | p      | p      | `2467153`         | `(0, 1, 3, 6, 2, 5, 4)` |
| 2 | p      | `-`    | `2463175`         | `(0, 1, 3, 2, 5, 6, 4)` |
| 3 | p      | `s`    | `2463157`         | `(6)(0, 1, 3, 2, 5, 4)` |
| 4 | `-`    | p      | `2456173`         | `(0, 1, 3, 5, 6, 2, 4)` |
| 5 | `-`    | `-`    | `2453167`         | `(6)(0, 1, 3, 2, 4)`    |
| 6 | `-`    | `s`    | `2453176`         | `(0, 1, 3, 2, 4)(5, 6)` |
| 7 | `s`    | p      | `2457163`         | `(0, 1, 3, 6, 2, 4)`    |
| 8 | `s`    | `-`    | `2453176`         | `(0, 1, 3, 2, 4)(5, 6)` |
| 9 | `s`    | `s`    | `2453167`         | `(6)(0, 1, 3, 2, 4)`    |

Some permutations in this list are duplicated:

* `--` reaches the same permutation as `ss` (ringers prefer `--`)
* `-s` reaches the same place as `s-` (ringers prefer `s-`)

... we can therefore immediately discard cases *6* and *9* above.

## Where can we reach?

Let's start with rounds and see where we can get to. I've created a `perm_from_row` helper function that converts to a [`sympy`](https://docs.sympy.org/latest/index.html) [`Permutation`](https://docs.sympy.org/latest/modules/combinatorics/permutations.html).

In [62]:
from stedman_searching.rows import perm_from_row

PAIR_END_ROWS = [
    '2467153',  # 1. pp
    '2463175',  # 2. p-
    '2463157',  # 3. ps
    '2456173',  # 4. -p
    '2453167',  # 5. --
    '2457163',  # 7. sp
    '2453176',  # 8. s-
]

PAIR_END_PERMS = [perm_from_row(row) for row in PAIR_END_ROWS]

PAIR_END_PERMS

[Permutation([1, 3, 5, 6, 0, 4, 2]),
 Permutation([1, 3, 5, 2, 0, 6, 4]),
 Permutation([1, 3, 5, 2, 0, 4, 6]),
 Permutation([1, 3, 4, 5, 0, 6, 2]),
 Permutation([1, 3, 4, 2, 0, 5, 6]),
 Permutation([1, 3, 4, 6, 0, 5, 2]),
 Permutation([1, 3, 4, 2, 0, 6, 5])]

A permutation can be combined with another one via multiplication. Let's apply these permutations to rounds and convert back to rows so we can check everything's working.

In [63]:
from stedman_searching.rows import row_from_perm
from sympy.combinatorics import Permutation

rounds = Permutation(6)

[row_from_perm(perm * rounds) for perm in PAIR_END_PERMS]

['2467153', '2463175', '2463157', '2456173', '2453167', '2457163', '2453176']

Change ringers should note that the `*` operator doesn't behave as we would expect when multiplying rows:

> The product of two permutations `p` and `q` is defined as their composition as functions, `(p*q)(i) = q(p(i))`.

Here's how we usually think of multiplying two rows:

```
  2413
× 3412
  ────
  1324
  ────
```

Here's how it works with `Permutation`:

In [64]:
Permutation.print_cyclic = False

row1 = Permutation(perm_from_row('2413'))
row2 = Permutation(perm_from_row('3412'))

print('Incorrect:', row_from_perm(row1 * row2))
print('Correct:',   row_from_perm(row2 * row1))

Incorrect: 4231
Correct: 1324


Let's generate the "six-pair ends" for a plain course of Stedman and check these match what we expect.

In [65]:
starting_row = perm_from_row('2314567')
plain_plain_pair = perm_from_row('2467153')

row = starting_row
for _ in range(7):
    row = plain_plain_pair * row
    print(row_from_perm(row))

3467251
4751326
7126435
1635742
6542173
5273614
2314567


After one six-pair we can reach any of seven rows when starting from rounds. Starting at each of those rows we can apply the same seven permutations in order to find the rows we can reach after two six-pairs.

In [66]:
[
    row_from_perm(second_perm * first_perm * starting_row)
    for first_perm in PAIR_END_PERMS
    for second_perm in PAIR_END_PERMS
]

['4751326',
 '4756312',
 '4756321',
 '4725316',
 '4726351',
 '4721356',
 '4726315',
 '4175326',
 '4176352',
 '4176325',
 '4127356',
 '4126375',
 '4125376',
 '4126357',
 '4157326',
 '4156372',
 '4156327',
 '4125376',
 '4126357',
 '4127356',
 '4126375',
 '4671325',
 '4675312',
 '4675321',
 '4627315',
 '4625371',
 '4621375',
 '4625317',
 '4167325',
 '4165372',
 '4165327',
 '4126375',
 '4125367',
 '4127365',
 '4125376',
 '4761325',
 '4765312',
 '4765321',
 '4726315',
 '4725361',
 '4721365',
 '4725316',
 '4176325',
 '4175362',
 '4175326',
 '4127365',
 '4125376',
 '4126375',
 '4125367']

Et voilà... 49 rows. Again we see that some of these rows are duplicated for the same reasons as above, but with pairs of calls occurring *between* the pairs of sixes:

```
  p-sp          p-s-
= ps-p        = ps--
```

... because `-s` = `s-`

```
  pssp          pss-
= p--p        = p---
```

... because `--` = `ss`

```
  --sp          --s-          s-sp          s-s-
= s--p        = s---        = ---p        = ----
```

... because these rules can be applied multiple times, e.g.: `s-s` → `ss-` → `---`. Accounting for the eight cases above we should have 41 distinct rows reachable within two six-pairs.

In [67]:
rows = [
    row_from_perm(second_perm * first_perm * starting_row)
    for first_perm in PAIR_END_PERMS
    for second_perm in PAIR_END_PERMS
]

len(set(rows))

35

It actually turns out that this isn't the case on seven bells (Stedman *Triples*). Individual bells traverse the row so quickly that they can be affected repeatedly by calls. On larger numbers of bells we see the expected result: 41 rows.

## Let's go deeper!

There are `5040` available rows on seven bells (7 *factorial* or `7!`), and so far we've only covered 36 of them. Let's go a little deeper. Let's continue using Python `set`s to eliminate duplicates, and build a function that can generate the set of results on the next stage.

In [89]:
def generate_next_rows(previous_rows):
    previous_perms = [perm_from_row(row) for row in previous_rows]
    return set([
        row_from_perm(pair_end_perm * previous_perm)
        for previous_perm in previous_perms
        for pair_end_perm in PAIR_END_PERMS
    ])

generate_next_rows(set(['1234567']))

{'2453167', '2453176', '2456173', '2457163', '2463157', '2463175', '2467153'}

In [91]:
all_rows = set()
rows_per_iteration = [set(['1234567'])]

for _ in range(9):
    previous_rows = rows_per_iteration[-1]
    next_rows = generate_next_rows(previous_rows)
    next_rows = next_rows - all_rows  # prune duplicates
    rows_per_iteration.append(next_rows)
    all_rows = all_rows | next_rows

print('rows reached:', len(all_rows))
print('furthest rows:', rows_per_iteration[-1])
[len(rows) for rows in rows_per_iteration]

rows reached: 5040
furthest rows: {'5162473', '5172436', '5162437', '5132476', '5172463', '5132467'}


[1, 7, 35, 135, 451, 905, 1429, 1443, 629, 6]

The result array shows the number of rows reachable at each iteration. We start with rounds, reach 7 rows after the first six-pair, 35 after the second, and then 135 after the third. This shows that:

* all 5040 rows are reachable within 9 six-pairs (18 sixes, or slightly over a course)
* 6 rows take exactly 9 six-pairs (these are the furthest from rounds)

This is a useful result, but we've not quite answered the question. The original problem was to find out how to get to rounds from a particular course end, but we've done the reverse. Luckily we can easily fix this by inverting:

In [92]:
[
    row_from_perm(~perm_from_row(row))
    for row in rows_per_iteration[-1]
]

['2475136', '2465173', '2465137', '2435176', '2475163', '2435167']